# Use Resources Wisely
With a Digital Research Alliance of Canada account, you have
access to many **high performance computing resources**:
* Compute clusters:
  * Béluga, Cedar, Graham, Narval, Niagara
* Storage :
  * Temporary, project, *nearline*

While being large, but still limited in size, these resources
should be used carefully by everyone in order to maximise the
amount of produced scientific results for themselves and for others.

## Goal - Writing a Proper Job Script for Slurm
The main goal of this chapter is to teach you how to
analyze your compute tasks in order to **determine the
required resources** to run tasks on compute clusters.
Each compute task will eventually be defined
in a **job script** to be submitted to the
[Slurm scheduler](https://slurm.schedmd.com/documentation.html).
Typically written in Bash commands, job scripts have:
* A [shebang](https://en.wikipedia.org/wiki/Shebang_(Unix))
  at the first line. For example: `#!/bin/bash`
* A header of `#SBATCH` options for the job's requirements.
  These options will be parsed at submission time by the
  [`sbatch` command](https://slurm.schedmd.com/sbatch.html)
* [Modules](https://docs.alliancecan.ca/wiki/Utiliser_des_modules/en)
  loaded before running the compute task
* The Bash commands that will be executed
  automatically on the reserved resources for the job

For example : [`scripts/mpi-hello.sh`](https://github.com/calculquebec/cip201-compute-systems/blob/main/scripts/mpi-hello.sh)

```Bash
cat scripts/mpi-hello.sh
```
```
#!/bin/bash
#SBATCH --ntasks=10
#SBATCH --mem-per-cpu=1000M
#SBATCH --time=0-00:10

module load gcc/9.3.0 &> /dev/null

mpirun printenv HOSTNAME OMPI_COMM_WORLD_RANK OMPI_COMM_WORLD_SIZE
```

Our documentation about job scripts starts at this page:
[Running jobs](https://docs.alliancecan.ca/wiki/Running_jobs)

## Analysing Compute Jobs on Your Computer
While a compute task is running on your computer,
you can monitor different metrics:
* CPU usage (and GPU usage, if applicable)
* Memory usage
* Storage access (IOPS, bandwidth)

### In Windows
* [Windows Tasks Manager](https://en.wikipedia.org/wiki/Task_Manager_(Windows))
* You can find it in two ways:
  * Look for *Task Manager* in the Start menu, or
  * With the keyboard shortcut Ctrl+Alt+Delete

![Windows 11 Task Manager screenshot](https://upload.wikimedia.org/wikipedia/en/a/ae/Windows_Task_Manager_screenshot.png)
_Image from Wikimedia_

### In macOS
* [Activity Monitor](https://support.apple.com/en-ca/guide/activity-monitor/actmntr1001/mac)
* To open the Activity Monitor:
  * Start it from the *Applications and Utilities* directory in macOS
  * Otherwise, use the Command+Space shortcut and start typing
    the first letters of "Activity Monitor" to find and select it

![Overview of Mac OS Activity Monitor](https://help.apple.com/assets/63FD404271C3B6058F265722/63FD404A71C3B6058F265732/en_US/07d6bea6dc9944ae58f6581297196752.png)
_Image from Apple Support_

### In Linux
In a Linux terminal, you can use:
* The [`top` command](https://man7.org/linux/man-pages/man1/top.1.html)
  (Press Q to quit)

![Screenshot of top](images/linux-top.png)

* The [`htop` command](https://man7.org/linux/man-pages/man1/htop.1.html)
  (Press Q to quit)

![Screenshot of htop](images/linux-htop.png)

## Analysing Compute Jobs on Clusters
On commence par **se connecter au noeud frontal** de la grappe :
```Bash
ssh login1
...
```
**Notes** :
* Pour accéder aux grappes de calcul en production, il vaut mieux
  utiliser [une paire de clés SSH](https://docs.alliancecan.ca/wiki/Using_SSH_keys_in_Linux/fr).
* [L'authentification multifacteur](https://docs.alliancecan.ca/wiki/Multifactor_authentication/fr)
  est maintenant offerte sur les grappes nationales.
  [Vidéo d'introduction ici](https://www.youtube.com/watch?v=ciycOUbchl8).
* Avec votre accès par défaut, vous avez un compte de calcul
  `def-*` de base qui vous permet de lancer des tâches.

To submit a job script, we use the
[`sbatch` command](https://slurm.schedmd.com/sbatch.html) :
```Bash
sbatch scripts/blastn-gen-seq.sh
```

And to monitor the status of a job, we use the
[`squeue` command](https://slurm.schedmd.com/squeue.html) :
```Bash
squeue -u $USER  # or 'sq'
```

### Resources Used by a Completed Job
With the [`sacct` command](https://slurm.schedmd.com/sacct.html),
we can get a detailed table of completed jobs since midnight:
```Bash
sacct
```

With the [`seff` command](https://docs.alliancecan.ca/wiki/Running_jobs#Completed_jobs),
we can get a short report about a single completed job.
This report includes the elapsed time, the total
CPU time and the maximum amount of memory used.
Two values of efficiency are given in percentages of total CPU
usage and maximum memory usage (compared to requested amounts).
```Bash
seff <Job_ID>
```

### Resources Used by a Running Job
Given some operations on a 3D matrix in the Python script
[`scripts/crunch.py`](https://github.com/calculquebec/cip201-compute-systems/blob/main/scripts/crunch.py) :

```Bash
cat scripts/crunch.py
```

While an interactive job is running, we can use the `top`
and `htop` commands to monitor resources being used:

```Bash
# Interactive job
salloc --cpus-per-task=4 --mem=8000M --time=0:15:0

cat scripts/crunch.sh

# Run with one CPU core
bash scripts/crunch.sh --cpu 1
top -u $USER  # Press Q to quit

# Run with four CPU cores
bash scripts/crunch.sh --cpu 4
htop -u $USER  # Press Q to quit

# Compare results
grep sec *.log

exit  # To go back to login1
```

If you use
[JupyterHub](https://docs.alliancecan.ca/wiki/JupyterHub)
to profile your codes, you can visualize in real time the
use of the _Machine Resources_ in the tab _GPU Dashboards_:

![GPU Dashboard for CPU](images/nv-dashboard_cpu.png)

#### **Exercise** - Checking Resources Used by a Running Job
While your job is running, you are allowed to connect by SSH to
the corresponding compute node in order to monitor your processes:
```Bash
cat scripts/inv-mat.sh
sbatch scripts/inv-mat.sh
```

Here are some general steps for job monitoring and validation:
* Identify on which node your job is running: `squeue -u $USER`
* Connect to that node with: `ssh <node_name>`
* Monitor the job execution with `top` or `htop`:
  * Are your processes running at **near 100%?**?
  * Are your parallel processes running at **near $n$ * 100%**,
    where $n$ is the number of reserved CPU cores for the job?
  * Does the **compute node** seem fully utilized?
* **Inspect results** in `time_inv.csv`
  * Identify any problem. If any, find the cause
  * Correct the code, the compilation, the script
    or the parameters used for the compute task
  * Resubmit the compute job and redo the above validation steps

### (Demo) Checking Resources Used by a Running GPU Job
```Bash
# Interactive job
salloc --cpus-per-task=4 --mem=8000M --time=0:15:0 --gres=gpu:1
```

* For Windows and Mac OS, you can install proprietary software
  that allows real time visualization of the GPU utilization.
  Please check the documentation of the GPU manufacturer for details
* In Linux, with an NVIDIA GPU, we first have the
  [`nvidia-smi` command](https://developer.nvidia.com/nvidia-system-management-interface)

```Bash
nvidia-smi
```

![Capture nvidia-smi](images/nvidia-smi.png)

* There is also the [`nvtop`](https://github.com/Syllo/nvtop) project
  that allows visualizing the use of one or many GPUs in a terminal:

```Bash
# Run with one GPU
bash scripts/crunch.sh --gpu
nvtop  # Press Q to quit
```
![Capture nvtop](https://raw.githubusercontent.com/Syllo/nvtop/master/screenshot/NVTOP_ex1.png)

```Bash
# Check the result
grep sec tg.log

exit  # To go back to login1
```

If you use
[JupyterHub](https://docs.alliancecan.ca/wiki/JupyterHub)
to profile your GPU software or code, you can visualize in real
time the use of _GPU Resources_ in the tab _GPU Dashboards_:

![GPU Dashboards for GPU](images/nv-dashboard_gpu.png)

#### **Exercise** - Testing `crunch.py` with One GPU
```Bash
cat    scripts/crunch-sbatch-1gpu.sh
sbatch scripts/crunch-sbatch-1gpu.sh

squeue -u $USER  # To check the status of the job
tail -24 $(ls slurm-* | tail -1)
```

#### Comparing the Speed of CPU Cores and a GPU
Avant d'utiliser massivement les GPUs d'une grappe de calcul, il faut
tout d'abord que l'application ou l'algorithme puisse démontrer une
"bonne performance" en utilisant plusieurs processeurs en parallèle.

Quelques définitions :
* **Temps écoulé** = temps d'exécution total que l'on perçoit et non le temps CPU
* **Accélération** = (temps avec un processeur) / (temps avec parallélisme)
* **Efficacité** = (Accélération) / (nombre de processeurs)

Le coût d'un noeud GPU étant cinq fois supérieur à celui d'un noeud
régulier, l'utilisation d'un seul GPU doit permettre une accélération
d'au moins cinq fois (5x) la vitesse de huit (8) à douze (12) processeurs.
* **Accélération** = (temps avec 8 à 12 processeurs) / (temps avec un accélérateur)

### Jobs Analysis via Cluster Portals
Béluga et Narval ont chacun un portail pour l'analyse des tâches :
* [https://portail.beluga.calculquebec.ca/](https://portail.beluga.calculquebec.ca/)
* [https://portail.narval.calculquebec.ca/](https://portail.narval.calculquebec.ca/)

## Estimating Required Compute Resources
### Target Efficiency of a Job
À coût d'essais et erreurs avec une **tâche de petite taille**,
la cible pour :
* **Le calcul** est une **efficacité d'au moins 90%**
  * Tâches séquentielles : il faut **optimiser les accès aux données**
    * Utiliser adéquatement les différents types de stockage
  * Tâches parallèles : il existe un **nombre maximal de processeurs**
    à utiliser pour respecter ce seuil :
    * Principe de [scalabilité](https://docs.alliancecan.ca/wiki/Scalability/fr)
      et [Loi d'Amdahl](https://fr.wikipedia.org/wiki/Loi_d%27Amdahl)
* **La mémoire-vive** est une consommation **de l'ordre de 80%**
  de ce qui est demandé à l'ordonnanceur Slurm

**Rappel** - vous pouvez obtenir ces pourcentages via les commandes
`sacct -X` (surtout pour obtenir les numéros de tâches) et `seff`.
Les valeurs à considérer sont :
* `CPU Utilized` et `CPU Efficiency`
* `Memory Utilized` et `Memory Efficiency`

#### **Exercise** - Job Efficiency
For some of your jobs listed by the following command:
```Bash
sacct -X
```
Get the `CPU Efficiency` and the `Memory Efficiency` with the command:
```Bash
seff <Job_ID>
```

### Extrapolating Required Compute Resources
La question qui se pose :
**en augmentant la ou les dimensions du problème**, quelles devraient
  être la durée du calcul et la consommation en mémoire-vive?
[Une analyse détaillée du code](https://fr.wikipedia.org/wiki/Analyse_de_la_complexit%C3%A9_des_algorithmes)
n'est pas nécessaire pour déterminer le type de calcul qui est fait :

* Vous pouvez **mesurer le temps d'exécution** (avec la
  commande `time`) en fonction de la taille du problème.
  En extrapolant les résultats, il serait possible de prévoir
  le comportement du programme sur une grappe de calcul.

```Bash
time -p sleep 2
```

* Vous pouvez considérer le **format des données en entrées** pour
  deviner l'ordre du calcul principal. 

### Data Size and Number of Files to Process
En plus du temps de calcul et de l'espace mémoire, il faut aussi
considérer **l'utilisation du stockage**. Les valeurs à tenir compte :
1. La **quantité** en octets (ou Go)
    * Peut servir à **estimer** l'utilisation de la mémoire-vive
    * Tenir compte de la taille du **stockage local rapide** pour
      optimiser les accès aux fichiers
1. Le **nombre** de fichiers à traiter
    * Considérer le **parallélisme de données**
    * **Multiplier la durée moyenne** du traitement d'un fichier par
      le nombre de fichiers pour estimer la durée d'une tâche
    * **Multiplier la taille moyenne** des fichiers par leur nombre
      pour estimer l'espace en mémoire-vive (par exemple : des images)
    * Utiliser le stockage rapide pour **optimiser les accès**
      aléatoires et nombreux

Pour obtenir le nombre de fichiers et la taille totale :
* **Sous Windows** : dans l'explorateur Windows (raccourcis clavier : Windows + E)
  * Sélectionner un dossier ou plusieurs fichiers
  * Bouton droit de la souris -> *Propriétés*

![Windows data properties](images/win-data-size.png)

* **Sous Mac OS** : dans *Finder*
  * Sélectionner un dossier ou plusieurs fichiers
  * Bouton droit de la souris -> *Get Info*
  * Autrement : avec l'affichage *Par liste*
    * [Activer *Calculer toutes les tailles*](https://www.solutionenligne.org/comment-afficher-taille-dossiers-fichiers-dans-finder-mac-os/)

* **In Linux** and on **compute clusters** :
  * The graphical environment can provide similar tools, but
    it depends on the Linux distribution and the chosen desktop
  * The command `du -sb DIRECTORY` (`s` for total sum, `b` for
    apparent size in bytes) recursively computes and displays
    the total size of used space in bytes. The apparent size is
    important to consider while transferring or backuping the data
  * The command `find DIRECTORY | wc -l` recursively counts
    and displays the number of files and subdirectories

```Bash
du -sb data
find data | wc -l
```

## Comparison of Compute Clusters
* About compute clusters:

| | [Béluga](https://docs.alliancecan.ca/wiki/B%C3%A9luga/en) | [Cedar](https://docs.alliancecan.ca/wiki/Cedar) | [Graham](https://docs.alliancecan.ca/wiki/Graham) | [Narval](https://docs.alliancecan.ca/wiki/Narval/en) | [Niagara](https://docs.alliancecan.ca/wiki/Niagara) |
|-----------------:|:----------:|:---------:|:---------:|:------------:|:----------:|
| **Availability** | March 2019 | June 2017 | June 2017 | October 2021 | April 2018 |
|         **City** |  Montréal  |  Burnaby  | Waterloo  |   Montréal   |   Toronto  |
|     **Province** |   Québec   |    B.C.   |  Ontario  |    Québec    |   Ontario  |

* Number of CPU cores (number of nodes * CPU cores per node) :

|     AMD/Intel CPU     | [Béluga](https://docs.alliancecan.ca/wiki/B%C3%A9luga/en) | [Cedar](https://docs.alliancecan.ca/wiki/Cedar) | [Graham](https://docs.alliancecan.ca/wiki/Graham) | [Narval](https://docs.alliancecan.ca/wiki/Narval/en) | [Niagara](https://docs.alliancecan.ca/wiki/Niagara) |
|----------------------:|:--------:|:--------:|:--------:|:---------:|:---------:|
|      Broadwell (avx2) |          | 724 * 32 | 983 * 32 |           |           |
|      Skylake (avx512) | 802 * 40 | 640 * 48 |          |           | 1548 * 40 |
| Cascade Lake (avx512) |          | 768 * 48 | 136 * 44 |           |  476 * 40 |
|      EPYC Rome (avx2) |          |          |          | 1181 * 64 |           |

| Memory per core | [Béluga](https://docs.alliancecan.ca/wiki/B%C3%A9luga/en) | [Cedar](https://docs.alliancecan.ca/wiki/Cedar) | [Graham](https://docs.alliancecan.ca/wiki/Graham) | [Narval](https://docs.alliancecan.ca/wiki/Narval/en) | [Niagara](https://docs.alliancecan.ca/wiki/Niagara) |
|-------:|:--------:|:---------:|:--------:|:---------:|:---------:|
|  2400M | 160 * 40 |           |          |           |           |
|  4000M |          | 1408 * 48 | 903 * 32 | 1145 * 64 |           |
|  4000M |          |  576 * 32 |          |           |           |
|  4400M |          |           | 136 * 44 |           |           |
|  4800M | 589 * 40 |           |          |           | 2024 * 40 |
|  8000M |          |   96 * 32 |  56 * 32 |           |           |
| 16000M |          |   24 * 32 |  24 * 32 |           |           |
| 19200M |  53 * 40 |           |          |           |           |
| 32000M |          |           |          |   36 * 64 |           |
| 48000M |          |   24 * 32 |   3 * 64 |           |           |
| 96000M |          |    4 * 32 |          |           |           |

* [Number of GPUs](https://docs.alliancecan.ca/wiki/Using_GPUs_with_Slurm)
  per cluster:

| GPU Model | [Béluga](https://docs.alliancecan.ca/wiki/B%C3%A9luga/en) | [Cedar](https://docs.alliancecan.ca/wiki/Cedar) | [Graham](https://docs.alliancecan.ca/wiki/Graham) | [Mist (Power9)](https://docs.scinet.utoronto.ca/index.php/Mist) | [Narval](https://docs.alliancecan.ca/wiki/Narval/en) |
|----------------:|:---:|:---:|:---:|:---:|:---:|
| NVIDIA P100 12G |     | 456 | 320 |     |     |
| NVIDIA P100 16G |     | 128 |     |     |     |
|   NVIDIA T4 16G |     |     | 144 |     |     |
| NVIDIA V100 16G | 688 |     |  54 |     |     |
| NVIDIA V100 32G |     | 768 |  16 | 216 |     |
| NVIDIA A100 40G |     |     |     |     | 636 |

* Other specifications :

| | [Béluga](https://docs.alliancecan.ca/wiki/B%C3%A9luga/en) | [Cedar](https://docs.alliancecan.ca/wiki/Cedar) | [Graham](https://docs.alliancecan.ca/wiki/Graham) | [Narval](https://docs.alliancecan.ca/wiki/Narval/en) | [Niagara](https://docs.alliancecan.ca/wiki/Niagara) |
|---------------------:|:----------:|:---------:|:----------:|:----------:|:----------:|
|         Fast network | InfiniBand | OmniPath  | InfiniBand | InfiniBand | InfiniBand |
|             Topology |  Fat Tree  | Fat Tree  |  Fat Tree  |  Fat Tree  | DragonFly+ |
|   Island size (core) |  640-1200  | 1024-1536 |    1024    | 3072-3584  |    17280   |
|      Blocking factor |  max 5:1   |  max 2:1  |  max 8:1   | max 4.7:1  |   max 2:1  |
| Job granularity (by) | core, GPU  | core, GPU | core, GPU  |  core, GPU |    node    |
|         Maximum time |   7 days   |  28 days  |   7 days   |   7 days   |   1 day    |

* Storage: all will be described in the last chapter.

## Key Points
* Prévoir les **paramètres d'une tâche Slurm**
  * Nombre de processeurs (CPU) et de noeuds de calcul
  * Nombre d'accélérateurs (GPU)
  * Quantité de mémoire-vive (RAM)
  * Temps du calcul (`JJ-H:M` ou `H:M:S`)
* Différents **outils pour surveiller** les ressources utilisées
  * `time` et autres bibliothèques de mesure du temps écoulé
  * `top`, `htop`, `nvtop`, `nvidia-smi`
  * `sacct`, `seff`
  * `du -sb`, `find | wc -l` et autres outils du système d'exploitation
* On vise une **efficacité de 90%** et plus pour les tâches CPU
  * L'accélération avec un accélérateur (GPU) doit être significative (>5x)
* Le choix de la grappe dépend des besoins de chaque type de calcul